<a href="https://colab.research.google.com/github/Ricardinho146/Alura_Challenge_Dados_2_Edit_Churn_Clientes/blob/main/Churn__Modelo_Preditor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importando as Bibliotecas

In [1]:
! pip install category_encoders


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 888.3 kB/s eta 0:00:00


In [2]:
! pip install -U Imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: Imbalanced-learn
    Found existing installation: imbalanced-learn 0.10.1
    Uninstalling imbalanced-learn-0.10.1:
      Successfully uninstalled imbalanced-learn-0.10.1


In [3]:
! pip install dython

In [4]:
! pip install streamlit --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.8 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
import category_encoders as ce
from category_encoders.one_hot import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, top_k_accuracy_score, confusion_matrix, classification_report, get_scorer_names
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn import metrics
from dython.nominal import associations
from dython.nominal import identify_nominal_columns
from sklearn.pipeline import Pipeline
import pickle
import plotly.express as px
import streamlit as st


Montando o dataframe

In [6]:
df = pd.read_json('https://challenge-data-science-3ed.s3.amazonaws.com/Telco-Customer-Churn.json')

df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


Expandindo os dados agrupados em json das colunas categóricas

In [7]:
# Expandindo os dados da coluna "customer" e inserindo colunas no DataFrame
expanded_data1 = pd.json_normalize(df['customer'])

# Expandindo os dados da coluna "phone" e inserindo colunas no DataFrame
expanded_data2 = pd.json_normalize(df['phone'])

# Expandindo os dados da coluna "internet" e inserindo colunas no DataFrame
expanded_data3 = pd.json_normalize(df['internet'])

# Expandindo os dados da coluna "internet" e inserindo colunas no DataFrame
expanded_data4 = pd.json_normalize(df['account'])
df = pd.concat([df, expanded_data1, expanded_data2, expanded_data3, expanded_data4], axis=1)

# Excluir as colunas originais 'customer','phone','internet'e'account'
df.drop(columns=['customer','phone','internet','account'], inplace=True)

print("Dataframe com os dados expandidos")
df.head()

Dataframe com os dados expandidos


,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


In [8]:
# Remover espaços em branco dos valores e substituir valores vazios por NaN condidos na coluna "Churn"
df['Churn'] = df['Churn'].str.replace(' ', '').replace('', pd.NA)
df['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [9]:
# Remover espaços em branco dos valores e substituir valores vazios por NaN
df['Charges.Total'] = df['Charges.Total'].str.replace(' ', '').replace('', pd.NA)

# Converter para float
df['Charges.Total'] = pd.to_numeric(df['Charges.Total'], errors='coerce')

# Verificando novamente tipo do Dado de cada coluna
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7043 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


Removendo os dados nulos que representam somente  0.1% ou 224 linhas do Dataframe.

In [10]:
df = df.dropna(axis=0)
# Verificando a quantidade de valores null ou NAN em cada coluna
print(df.isnull().sum())

customerID          0
Churn               0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
Charges.Monthly     0
Charges.Total       0
dtype: int64


In [11]:
lb = LabelEncoder()
df['Churn']  = lb.fit_transform(df['Churn'])
df.head()

<ipython-input-11-fddc9edacfc2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Churn']  = lb.fit_transform(df['Churn'])


,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,0,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30
1,0003-MKNFE,0,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40
2,0004-TLHLJ,1,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,1,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,1,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40


In [12]:
x = df.drop("Churn", axis =1)
y = df.Churn

Testando o Modelo sem One Hot Encoding

In [13]:
# Removendo a coluna "customerID" do dataframe pois não faz sentido para análise

df_tst = df.drop("customerID", axis =1)

# Gerando uma lista com as colunas de dados categóricos
dados_categóricos =identify_nominal_columns(df_tst)
dados_categóricos

['gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod']

In [88]:
le = LabelEncoder()

for coluna in dados_categóricos:
    df_tst[coluna] = le.fit_transform(df_tst[coluna])

df_tst.head()


,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0,0,0,1,1,9,1,0,0,0,2,0,2,2,0,1,1,3,65.6,593.30
1,0,1,0,0,0,9,1,2,0,0,0,0,0,0,2,0,0,3,59.9,542.40
2,1,1,0,0,0,4,1,0,1,0,0,2,0,0,0,0,1,2,73.9,280.85
3,1,1,1,1,0,13,1,0,1,0,2,2,0,2,2,0,1,2,98.0,1237.85
4,1,0,1,1,0,3,1,0,1,0,0,0,2,2,0,0,1,3,83.9,267.40


In [15]:
x_1 = df_tst.drop("Churn", axis =1)
y_1 = df_tst.Churn

In [16]:
ros = RandomOverSampler(sampling_strategy = 1)

x_ros_1, y_ros_1 = ros.fit_resample(x_1,y_1)

y_ros_1.value_counts()

0    5163
1    5163
Name: Churn, dtype: int64

In [17]:
x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(x_ros_1, y_ros_1, test_size = 0.2, random_state = 42 )

x_train_1.shape, x_test_1.shape, y_train_1.shape, y_test_1.shape

((8260, 19), (2066, 19), (8260,), (2066,))

In [18]:
modelo_tst = RandomForestClassifier( random_state = 42)

modelo_tst.fit(x_train_1, y_train_1)

y_pred_1 = modelo_tst.predict(x_test_1)

print('Scores de Classificação: \n \n',classification_report(y_test_1, y_pred_1))


Scores de Classificação: 
 
               precision    recall  f1-score   support

           0       0.94      0.84      0.89      1062
           1       0.85      0.95      0.89      1004

    accuracy                           0.89      2066
   macro avg       0.89      0.89      0.89      2066
weighted avg       0.90      0.89      0.89      2066



In [19]:
k_vs_score2 = []
score_list2 = []

for k in range(1, 19, 1):
    # Configurando todas as variáveis
    seletor_variaveis = SelectKBest(score_func=f_classif, k=k)

    # Realizando o fit das variáveis e os valores target
    Xtreino2 = seletor_variaveis.fit_transform(x_train_1, y_train_1)
    Xteste2 = seletor_variaveis.transform(x_test_1)

    modelo_RFC = RandomForestClassifier(n_estimators = 50, min_samples_split = 3, min_samples_leaf = 1,
                                         max_depth = 20, bootstrap = False,  n_jobs=-1, random_state= 42)
    modelo_RFC.fit(Xtreino2, y_train_1)

    Y_predict_selecionado = modelo_RFC.predict(Xteste2)

    score = metrics.f1_score(y_test_1, Y_predict_selecionado)

    k_vs_score2.append(k)
    score_list2.append(score)

In [20]:
px.line( x = k_vs_score2, y = score_list2, template = 'plotly_dark',
        title= "Seleção de Variáveis - Qtd. Variáveis X F1-Score - RandomForestClassifier", markers="*")

Gerando lista com os dados preditores (Categóricos + Numérios):

In [43]:
# Criando as listas
Gender = df['gender'].unique()
SeniorCitizen = df['SeniorCitizen'].unique()
Partner = df['Partner'].unique()
Dependents = df['Dependents'].unique()
Tenure = df['tenure'].unique()
PhoneService = df['PhoneService'].unique()
MultipleLines = df['MultipleLines'].unique()
InternetService = df['InternetService'].unique()
OnlineSecurity = df['OnlineSecurity'].unique()
OnlineBackup = df['OnlineBackup'].unique()
DeviceProtection = df['DeviceProtection'].unique()
TechSupport = df['TechSupport'].unique()
StreamingTV = df['StreamingTV'].unique()
StreamingMovies = df['StreamingMovies'].unique()
Contract = df['Contract'].unique()
PaperlessBilling = df['PaperlessBilling'].unique()
PaymentMethod = df['PaymentMethod'].unique()
ChargesMonthly = df['Charges.Monthly'].unique()
ChargesTotal = df['Charges.Total'].unique()


# Realizando o print das listas

print('Gender: ',Gender)
print('SeniorCitizen: ',SeniorCitizen)
print('Partner: ',Partner)
print('Dependents: ',Dependents)
print('tenure: ',Tenure.min(), Tenure.max())
print('PhoneService: ',PhoneService)
print('MultipleLines: ',MultipleLines)
print('InternetService: ',InternetService)
print('OnlineSecurity: ',OnlineSecurity)
print('OnlineBackup: ',OnlineBackup)
print('DeviceProtection: ',DeviceProtection)
print('TechSupport: ',TechSupport)
print('StreamingTV: ',StreamingTV)
print('StreamingMovies: ',StreamingMovies)
print('Contract: ',Contract)
print('PaperlessBilling: ',PaperlessBilling)
print('PaymentMethod: ',PaymentMethod)
print('ChargesMonthly: ',ChargesMonthly.min(), ChargesMonthly.max())
print('ChargesTotal: ',ChargesTotal.min(), ChargesTotal.max())



Gender:  ['Female' 'Male']
SeniorCitizen:  [0 1]
Partner:  ['Yes' 'No']
Dependents:  ['Yes' 'No']
tenure:  1 72
PhoneService:  ['Yes' 'No']
MultipleLines:  ['No' 'Yes' 'No phone service']
InternetService:  ['DSL' 'Fiber optic' 'No']
OnlineSecurity:  ['No' 'Yes' 'No internet service']
OnlineBackup:  ['Yes' 'No' 'No internet service']
DeviceProtection:  ['No' 'Yes' 'No internet service']
TechSupport:  ['Yes' 'No' 'No internet service']
StreamingTV:  ['Yes' 'No' 'No internet service']
StreamingMovies:  ['No' 'Yes' 'No internet service']
Contract:  ['One year' 'Month-to-month' 'Two year']
PaperlessBilling:  ['Yes' 'No']
PaymentMethod:  ['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']
ChargesMonthly:  18.25 118.75
ChargesTotal:  18.8 8684.8



Gerando lista com as colunas utilizadas no Data Frame:


In [22]:
list(df_tst.columns)

['Churn',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'Charges.Monthly',
 'Charges.Total']

In [23]:
df.describe()

,Churn,SeniorCitizen,tenure,Charges.Monthly,Charges.Total
count,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000
mean,0.265785,0.162400,32.421786,64.798208,2283.300441
std,0.441782,0.368844,24.545260,30.085974,2266.771362
min,0.000000,0.000000,1.000000,18.250000,18.800000
25%,0.000000,0.000000,9.000000,35.587500,401.450000
50%,0.000000,0.000000,29.000000,70.350000,1397.475000
75%,1.000000,0.000000,55.000000,89.862500,3794.737500
max,1.000000,1.000000,72.000000,118.750000,8684.800000


Criando o Site com o streamlit

# Verificar todas as colunas que estão sendo carregadas pois no modelo atual estão sendo carregadas somente os dados categórigos, após isso aplicar o label encoding dentro do modelo streamlit


In [86]:

%%writefile app.py

import pandas as pd
import numpy as np
from dython.nominal import associations
from dython.nominal import identify_nominal_columns
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import streamlit as st

df = pd.read_json('https://challenge-data-science-3ed.s3.amazonaws.com/Telco-Customer-Churn.json')
# Expandindo os dados da coluna "customer" e inserindo colunas no DataFrame
expanded_data1 = pd.json_normalize(df['customer'])

# Expandindo os dados da coluna "phone" e inserindo colunas no DataFrame
expanded_data2 = pd.json_normalize(df['phone'])

# Expandindo os dados da coluna "internet" e inserindo colunas no DataFrame
expanded_data3 = pd.json_normalize(df['internet'])

# Expandindo os dados da coluna "internet" e inserindo colunas no DataFrame
expanded_data4 = pd.json_normalize(df['account'])
df = pd.concat([df, expanded_data1, expanded_data2, expanded_data3, expanded_data4], axis=1)

# Excluir as colunas originais 'customer','phone','internet'e'account'
df.drop(columns=['customer','phone','internet','account'], inplace=True)

# Remover espaços em branco dos valores e substituir valores vazios por NaN condidos na coluna "Churn"
df['Churn'] = df['Churn'].str.replace(' ', '').replace('', pd.NA)

# Remover espaços em branco dos valores e substituir valores vazios por NaN
df['Charges.Total'] = df['Charges.Total'].str.replace(' ', '').replace('', pd.NA)

# Converter para float
df['Charges.Total'] = pd.to_numeric(df['Charges.Total'], errors='coerce')

# Removendo dados nulos
df = df.dropna(axis=0)

lb = LabelEncoder()
df['Churn']  = lb.fit_transform(df['Churn'])

# Removendo a coluna "customerID" do dataframe pois não faz sentido para análise

df_tst = df.drop("customerID", axis =1)

# Gerando uma lista com as colunas de dados categóricos
dados_categóricos =identify_nominal_columns(df_tst)


le = LabelEncoder()

for coluna in dados_categóricos:
    df_tst[coluna] = le.fit_transform(df_tst[coluna])

df_tst.head()

x_1 = df_tst.drop("Churn", axis =1)
y_1 = df_tst.Churn

ros = RandomOverSampler(sampling_strategy = 1)

x_ros_1, y_ros_1 = ros.fit_resample(x_1,y_1)

x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(x_ros_1, y_ros_1, test_size = 0.2, random_state = 42 )


modelo_tst = RandomForestClassifier( random_state = 42)

modelo_tst.fit(x_train_1, y_train_1)

y_pred_1 = modelo_tst.predict(x_test_1)

import streamlit as st


st.title('# My First Data App')
st.write('## Churn Predict Model with RandomForestClassifier')

gender = st.selectbox("Select Gender: ", ['Female','Male'])
SeniorCitizen = st.selectbox("Select SeniorCitizen: ", ['0','1'])
partner = st.selectbox("Select Partner: ", ['Yes','No'])
dependents = st.selectbox("Select Dependents: ", ['Yes','No'])
Tenure = st.slider("Select Tenure: ", 1, 72)
phoneService = st.selectbox("Select PhoneService: ", ['Yes','No'])
multipleLines = st.selectbox("Select MultipleLines: ", ['No','Yes','No phone service'])
internetservice = st.selectbox("Select InternetService: ", ['DSL','Fiber optic','No'])
onlinesecurity = st.selectbox("Select OnlineSecurity: ", ['No','Yes','No internet service'])
onlinebackup = st.selectbox("Select OnlineBackup: ", ['Yes','No','No internet service'])
deviceProtection = st.selectbox("Select DeviceProtection: ", ['No','Yes','No internet service'])
techSupport = st.selectbox("Select TechSupport: ", ['Yes','No','No internet service'])
streamingTV = st.selectbox("Select StreamingTV: ", ['Yes','No','No internet service'])
StreamingMovies = st.selectbox("Select StreamingMovies: ", ['No', 'Yes', 'No internet service'])
contract = st.selectbox("Select Contract: ", ['One year','Month-to-month','Two year'])
paperlessbilling = st.selectbox("Select PaperlessBilling: ", ['Yes','No'])
paymentMethod = st.selectbox("Select PaymentMethod: ", ['Mailed check','Electronic check','Credit card (automatic)','Bank transfer (automatic)'])
ChargesMonthly= st.slider("Select ChargesMonthly ", 18.25, 118.75)
ChargesTotal= st.slider("Select ChargesTotal",18.8, 8684.8)


def predict():
    input_data = {
        'gender': gender,
        'SeniorCitizen': SeniorCitizen,
        'Partner': partner,
        'Dependents': dependents,
        'tenure': Tenure,
        'PhoneService': phoneService,
        'MultipleLines': multipleLines,
        'InternetService': internetservice,
        'OnlineSecurity': onlinesecurity,
        'OnlineBackup': onlinebackup,
        'DeviceProtection': deviceProtection,
        'TechSupport': techSupport,
        'StreamingTV': streamingTV,
        'StreamingMovies': StreamingMovies,
        'Contract': contract,
        'PaperlessBilling': paperlessbilling,
        'PaymentMethod': paymentMethod,
        'Charges.Monthly': ChargesMonthly,
        'Charges.Total': ChargesTotal
    }

    # Criar um DataFrame a partir dos dados de entrada do usuário
    input_df = pd.DataFrame([input_data])

    # Mapear 'Yes' para 1 e 'No' para 0 nos dados booleanos
    boolean_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
    input_df[boolean_cols] = input_df[boolean_cols].replace({'Yes': 1, 'No': 0})

    # Mapear 'Female' para 1 e 'Male' para 0 na coluna 'gender'
    input_df['gender'] = input_df['gender'].map({'Female': 1, 'Male': 0})

    # Criar um LabelEncoder separado para os outros dados categóricos
    le_input = LabelEncoder()

    # Converter outros dados categóricos usando LabelEncoder
    for w in dados_categóricos:
        if w not in boolean_cols + ['gender']:
            input_df[w] = le_input.fit_transform(input_df[w])

    # Mostrar os dados de entrada
    st.write('Input Data:')
    st.write(input_df)

    # Fazer a previsão
    prediction = modelo_tst.predict(input_df)

    # Calcular a acurácia usando os dados de teste
    accuracy = modelo_tst.score(x_test_1, y_test_1)

    st.write(f'Model Accuracy: {accuracy:.2%}')

    if prediction[0] == 1:
        st.success('Non-Quitting Customer :thumbsup:')
    else:
        st.error('Customer Abandonment :thumbsdown:')

trigger = st.button('Predict', on_click=predict)







Overwriting app.py


In [85]:
!streamlit run app.py & npx localtunnel --port 8501

[..................] - rollbackFailedOptional: verb npm-session 9dbb471f3723c4e


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.23.204:8501

npx: installed 22 in 2.846s
your url is: https://poor-mirrors-hug.loca.lt
  Stopping...
^C
